In [1]:
from ppadb.client import Client
from PIL import Image
import numpy as np
import time
import math

abd = Client(host="127.0.0.1", port=5037)
devices = abd.devices()

if (len(devices) == 0):
    print("no devices")
else:
    device = devices[0] #Assume first device is the device needed

In [3]:
def turnScreenPixelsToArray():
    #This function takes a device screencap and converts it into a numpy array.  Returning a image that is in (y,x) coordinates.
    # Also known as (height, width)
    
    image = device.screencap()
    with open("game.png", 'wb') as file:
        file.write(image)
        
    image = Image.open('game.png')
    image = np.array(image, dtype=np.uint8)  #convert image into numpy array for easily accessing pixel data. 
                                             # In (y,x) coordinate system instead of (x,y)
        
    return image

In [201]:
def checkForCorrectColor(currentPixel):
    #Checks if the pixel is rgb color for the rims on the screen
    if(currentPixel[0] == 216 and currentPixel[1] == 79 and currentPixel[2] == 44):
        #Red rim pixel
        return True
    if(currentPixel[0] == 170 and currentPixel[1] == 170 and currentPixel[2] == 170):
        #Gray rim pixel
        return True
    return False

    
def findMiddleOfHoops(rgbImage):
    #This function works by finding every singlepixel on the rims of the hoop and averaging the coordinates to 
    #find the middle of the hoop.
    
    aimHoopLocation = [0,0]  #Both of these are given in x,y coordinates
    ballHoopLocation = [0,0]
    numOfPixelsInFirstHoop = 0  #Number of red pixels found in first hoop
    numOfPixelsInSecondHoop = 0  #Number of red pixels found in second hoop
    numOfHoopsCompleted = 0  #Number of hoops all the pixels for have been found
    foundHoop = False;  #Tells the program a hoop has been found
    rowHasRim = False;  #Tells the program the row has rim pixels
        
    for i in range(len(rgbImage)):
        if((not rowHasRim) and foundHoop):
            #Move to next hoop if at end of current hoop
            numOfHoopsCompleted += 1
            foundHoop = False
        
        if(numOfHoopsCompleted == 2):
            #Exit loop after two hoops are found
            break;
        
        rowHasRim = False
        for j in range(len(rgbImage[0])): #Loop through width of screen
            currentPixel = rgbImage[i][j]
            
            if(checkForCorrectColor(currentPixel)):
                #Curent pixel is the correct rgb color for the rims on the screen
                if(not foundHoop):
                    #Found a hoop
                    foundHoop = True
                
                if(not rowHasRim):
                    #Current row has part of the rim
                    rowHasRim = True
                
                if(numOfHoopsCompleted == 0):
                    #Adds pixel's coordinates to the hoop to aim at
                    aimHoopLocation[0] += j
                    aimHoopLocation[1] += i
                    numOfPixelsInFirstHoop += 1
                else:
                    #Adds pixel's coordinates of the ball hoop
                    ballHoopLocation[0] += j
                    ballHoopLocation[1] += i
                    numOfPixelsInSecondHoop += 1
            
    
    #This is where the coodinates are averaged with the number of pixels found
    aimHoopLocation[0] = int(aimHoopLocation[0] / numOfPixelsInFirstHoop)
    aimHoopLocation[1] = int(aimHoopLocation[1] / numOfPixelsInFirstHoop) + 82
    ballHoopLocation[0] = int(ballHoopLocation[0] / numOfPixelsInSecondHoop)
    ballHoopLocation[1] = int(ballHoopLocation[1] / numOfPixelsInSecondHoop) + 82
    
    return aimHoopLocation, ballHoopLocation

In [186]:
def calcPullback(aimHoopLocation, ballHoopLocation):
    #This function calculates how far to pull the ball and what direction.  It works by throwing the ball at a set y 
    # velocity and using that to calculate the required x velocity.  This is then used to find the angle to drag the ball.
    
    yVel = -2800 #How hard to throw the ball in the y direction in
    accel = 2872  #This is the acceleration due to gravity in pixels/sec^2
    delX = aimHoopLocation[0] - ballHoopLocation[0]  #change in x position
    delY = aimHoopLocation[1] - ballHoopLocation[1]  #change in y position
    
    #Calculate the final y velocity before hitting the hoop to be aimed at
    yVelFinal = math.sqrt(yVel ** 2 + 2 * accel * delY)
    
    #Calculate time in air using y final velocity
    time = (yVelFinal - yVel) / accel
    
    #Calculate initial x velocity with time in air
    xVel = delX / time
    
    #Calculate throwing angle with respect to the vertical
    throwAngle = math.atan(xVel/yVel)
    print(math.degrees(throwAngle))
    
    #Calculate how far to pull back on screen
    yPullBack = -700
    xPullBack = math.tan(throwAngle) * yPullBack
    
    return xPullBack, yPullBack
    

In [198]:
def makeShot():
    #This function holds the simple ai.  It is the main part of the program.
    
    image = turnScreenPixelsToArray() #This image is in y,x coordinates not x,y
    
    screenHeight = len(image)  #Get height of screen for looping through pixels
    screenWidth = len(image[0])  #Get width of screen for looping through pixels
    
    #This next loop goes through each of the pixels and omits the opacity value
    rgbImage = np.zeros((screenHeight, screenWidth, 3))
    for i in range(screenHeight):
        for j in range(screenWidth):
            rgbImage[i][j] = image[i][j][:3]
    
    rgbImage = rgbImage[1000:2200] #Take only the rows that matter on the screen
    print(rgbImage[258][1192])
    
    #Find Hoop Locations
    aimHoopLocation, ballHoopLocation = findMiddleOfHoops(rgbImage) #These locations are in x,y coordinates now
    
    #Calculate the distance in pixels to pull the ball to throw it into the hoop
    xPullBack, yPullBack = calcPullback(aimHoopLocation, ballHoopLocation)
    
    #Issue swipe command to the phone with given pull back distances
    #device.shell("input touchscreen swipe x1 y1 x2 y2 dur")
    command = "input touchscreen swipe 700 800 " + str(700 - xPullBack) + " " + str(800 - yPullBack) + " 1500"
    print(command)
    device.shell(command)
    
    print(xPullBack)
    print(yPullBack)
    print(aimHoopLocation)
    print(ballHoopLocation)

In [207]:
for i in range(20):
    makeShot()
    time.sleep(4)

[232. 232. 232.]
6.064663044892568
input touchscreen swipe 700 800 774.3717950488787 1500 1500
-74.37179504887868
-700
[366, 247]
[906, 598]
[223. 223. 223.]
0.568248998754817
input touchscreen swipe 700 800 706.9426988361625 1500 1500
-6.942698836162561
-700
[764, 257]
[818, 272]
[232. 232. 232.]
-0.0
input touchscreen swipe 700 800 700.0 1500 1500
0.0
-700
[1058, 519]
[1058, 568]
[232. 232. 232.]
-7.508862263058869
input touchscreen swipe 700 800 607.7330998479356 1500 1500
92.26690015206447
-700
[1058, 642]
[393, 1025]
[232. 232. 232.]
7.3742475174652204
input touchscreen swipe 700 800 790.5942121802199 1500 1500
-90.59421218021983
-700
[418, 558]
[1058, 1024]
[232. 232. 232.]
-0.010590399508180356
input touchscreen swipe 700 800 699.870613860163 1500 1500
0.12938613983702046
-700
[1032, 466]
[1031, 515]
[232. 232. 232.]
-0.010590399508180356
input touchscreen swipe 700 800 699.870613860163 1500 1500
0.12938613983702046
-700
[1032, 466]
[1031, 515]
[232. 232. 232.]
-0.01059039950818

KeyboardInterrupt: 

In [ ]:
def findMiddleOfHoops(rgbImage):
    #This function works by finding every single red pixel on the rims of the hoop and averaging the coordinates to 
    #find the middle of the hoop.  This one is used for the first throw of the game
    
    aimHoopLocation = [0,0]  #Both of these are given in x,y coordinates
    ballHoopLocation = [0,0]
    numOfPixelsInFirstHoop = 0  #Number of red pixels found in first hoop
    numOfPixelsInSecondHoop = 0  #Number of red pixels found in second hoop
    numOfHoopsCompleted = 0  #Number of hoops all the pixels for have been found
    foundHoop = False;  #Tells the program a hoop has been found
    rowHasRed = False;  #Tells the program the row has red pixels
        
    for i in range(len(rgbImage)):
        if((not rowHasRed) and foundHoop):
            #Move to next hoop if at end of current hoop
            numOfHoopsCompleted += 1
            foundHoop = False
        
        if(numOfHoopsCompleted == 2):
            #Exit loop after two hoops are found
            break;
        
        rowHasRed = False
        for j in range(len(rgbImage[0])): #Loop through width of screen
            currentPixel = rgbImage[i][j]
            if(currentPixel[0] == 216 and currentPixel[1] == 79 and currentPixel[2] == 44):
                #Checks if the pixel is within the rgb threshold for the red rims on the screen
                if(not foundHoop):
                    #Found a hoop
                    foundHoop = True
                
                if(not rowHasRed):
                    #Current row has red
                    rowHasRed = True
                
                if(numOfHoopsCompleted == 0):
                    #Adds pixel's coordinates to the hoop to aim at
                    aimHoopLocation[0] += j
                    aimHoopLocation[1] += i
                    numOfPixelsInFirstHoop += 1
                else:
                    #Adds pixel's coordinates of the ball hoop
                    ballHoopLocation[0] += j
                    ballHoopLocation[1] += i
                    numOfPixelsInSecondHoop += 1
            
    
    #This is where the coodinates are averaged with the number of pixels found
    aimHoopLocation[0] = int(aimHoopLocation[0] / numOfPixelsInFirstHoop)
    aimHoopLocation[1] = int(aimHoopLocation[1] / numOfPixelsInFirstHoop) + 82
    ballHoopLocation[0] = int(ballHoopLocation[0] / numOfPixelsInSecondHoop)
    ballHoopLocation[1] = int(ballHoopLocation[1] / numOfPixelsInSecondHoop) + 82
    
    return aimHoopLocation, ballHoopLocation